# Notebook 4.1: Support Vector Machines

### Machine Learning Basic Module
Florian Walter, Tobias Jülg, Pierre Krack

Please obey the following implementation and submission guidelines.

## General Information About Implementation Assignments
We will use the Jupyter Notebook for our implementation exercises. The task description will be provided in the notebook. The code is also run in the notebook. However, the implementation itself is done in additional files which are imported in the notebook. Please do not provide any implementation that you want to be considered for correction in this notebook, but only in Python files in the marked positions. A content of a python file could for example look similar as shown below:
```python
def f():
    ########################################################################
    # YOUR CODE
    # TODO: Implement this function
    ########################################################################
    pass
    ########################################################################
    # END OF YOUR CODE
    ########################################################################
```
To complete the exercise, remove the `pass` command and only use space inside the `YOUR CODE` block to provide a solution. Other lines within the file may not be changed in order to deliver a valid submission.

## General Information About Theory Assignments
This Jupyter Notebook also includes one or more theory assignments. The theory assignments have to be solved ina PDF file.

You can either typeset your solution in $\LaTeX$/Word or hand-in a digital written or scanned solution.


### Imports

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from svm import solve_dual_svm, compute_weights_and_bias, plot_data_with_hyperplane_and_support_vectors, ALPHA_TOL

## Binary SVM Classifier (hard margin)
In this exercise we want to implement our own binary SVM classifier.
We will start by creating a synthetic dataset. Run the cell below to create and visualize the dataset.

In [ ]:
N = 200  # number of samples
D = 2  # number of dimensions
C = 2  # number of classes
seed = 1234  # for reproducible experiments

# alpha_tol = 1e-4 # threshold for choosing support vectors

X, y = make_blobs(n_samples=N, n_features=D, centers=C, random_state=seed)
y[y == 0] = -1  # it is more convenient to have {-1, 1} as class labels (instead of {0, 1})
y = y.astype(float)
plt.figure(figsize=[10, 8])
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

In this week's learning unit you should have learned that the SVM dual problem can be formulated as a Quadratic programming (QP) problem by solving the Lagrange dual problem. For a dataset with $N$ data points

$$
\mathcal{D} = \{(\mathbf{x}_i, y_i)\}_{i=1}^N
$$

with features $\mathbf{x}_i \in \mathbb{R}^D$ and binary labels $y_i \in \{-1, 1\}$,
the SVM's dual problem is given as

$$
\begin{aligned}
\max_{\mathbf{\alpha}} &\quad g(\mathbf{\alpha}) = \max_{\mathbf{\alpha}} \sum_{i=1}^N \alpha_i - \frac{1}{2} \sum_{i=1}^N \sum_{j=1}^N \alpha_i \alpha_j y_i y_j \mathbf{x}_i^T \mathbf{x}_j = \\
\text{subj.} &\quad \sum_{i=1}^N \alpha_i y_i = 0 \\
& \alpha_i \ge 0 \quad \text{for} \quad i = 1, \dots, N
\end{aligned}
$$ 

where $\mathbf{\alpha}\in\mathbb{R}_{\ge 0}^N$ is the vector of Lagrange multipliers.

> **Task 1** Show that the dual function $g(\alpha)$ can be written as
> 
> $$g(\alpha) = \frac{1}{2}\alpha^T\mathbf{Q}\alpha + \alpha^T \mathbf{1}_N$$
> 
> by providing the matrix $\mathbf{Q}$ using the vector of labels $\mathbf{y}\in\mathbb{R}^N$ and feature matrix $\mathbf{X}\in\mathbb{R}^{N\times D}$ such that terms are equal. Denote the element-wise product between two matrices (in case you want to use it) by $\odot$ (also known as Hadamard product or Schur product).

We will solve the dual problem using a QP solver from the [`CVXOPT` library](https://cvxopt.org/) (a python library for convex optimization).

The `CVXOPT` solver expects its QP problem to be in the following form:
$$
\begin{aligned}
\underset{\mathbf{x}}{\min}\quad
    &\frac{1}{2}\mathbf{x}^T \mathbf{P} \mathbf{x} + \mathbf{q}^T \mathbf{x} \\
\text{subj.}\quad
    &\mathbf{G}\mathbf{x} \le \mathbf{h}\\
    &\mathbf{A}\mathbf{x} = \mathbf{b}
\end{aligned}
$$

> **Task 2** Formulate the SVM dual problems as a QP of this form and solve it using `CVXOPT`, i.e. specify the matrices $\mathbf{P}, \mathbf{G}, \mathbf{A}$ and vectors $\mathbf{q}, \mathbf{h}, \mathbf{b}$. Implement the `solve_dual_svm()` function in the `svm.py` file.

Having obtained the optimal $\alpha^*$ using our QP solver, we can compute the parameters defining the separating hyperplane.

Recall, that from the optimality condition, the weights $w$ are a linear combination of the training samples,
$$
w = \sum_{i=1}^{N} \alpha_i^* y_i x_i
$$

From the complementary slackness condition $\alpha_i^* f_i(\theta^*) = 0$ we can easily recover the bias.

When we take any vector $x_i$ for which $\alpha_i \neq 0$. The corresponding constraint $f_i(w, b)$ must be zero and thus we have
$$
w^T x_i + b = y_i.
$$

Solving this for $b$ yields the bias
$$
b = y_i - w^T x_i
$$

> **Task 3** Given this information, implement the `compute_weights_and_bias()` function in the `svm.py` file.


Run the cell below to visualize the decision boundary and the support vectors. The code should work without any errors if you have implemented the functions correctly. Check the printed output of the cell with the reference solution given below. If your implementation is correct, the output should be the same.

In [ ]:
alpha = solve_dual_svm(X, y)
w, b = compute_weights_and_bias(alpha, X, y)
print("w =", w)
print("b =", b)
print("support vectors:", np.arange(len(alpha))[alpha > ALPHA_TOL])
plot_data_with_hyperplane_and_support_vectors(X, y, alpha, w, b)
plt.show()

The reference solution is

    w = array([0.73935606 0.41780426])
    
    b = 0.91993713

Indices of the support vectors are
    
    [ 78 134 158 182]

# Kernels
Consider the following two plots.
On the left is the original data that we want to separate linearly.
On the right is the augmented data which comes from the feature mapping function $\phi((x_0, x_1)) = (x_0, x_1, x_0^2, x_1^2)$

In [ ]:
rng = np.random.default_rng(seed=seed)
data = rng.uniform(low=-1, high=1, size=(2, 100))
index_class_1 = np.logical_and(np.abs(data[0]) < 0.4, np.abs(data[1]) < 0.4)
class_1 = data[:, index_class_1]
class_2 = data[:, np.logical_not(index_class_1)]
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1)
ax.scatter(class_1[0], class_1[1])
ax.scatter(class_2[0], class_2[1])
ax = fig.add_subplot(1, 2, 2, projection='3d')
def phi(x):
    return np.array([x[0], x[1], x[0]**2 + x[1]**2])
class_1 = phi(class_1)
class_2 = phi(class_2)
ax.scatter(class_1[0], class_1[1], class_1[2])
ax.scatter(class_2[0], class_2[0], class_2[2])
plt.show()

This mapping allows us to make the data linearly separable in the feature space.
> **Task 4** Find a kernel function which implicitly uses the feature mapping $\phi((x_0, x_1)) = (x_0, x_1, x_0^2 + x_1^2)^T$ (plotted above).
$$
\newcommand{\defeq}{\stackrel{\text{def}}{=}}
\newcommand{\x}{\textrm{x}}
\newcommand{\xprime}{\x^\prime}
$$



In this case, finding a feature mapping that works is quite simple and we can even plot our mapping to verify if it works.
When the data becomes more complex choosing an appropriate feature mapping becomes considerably harder.
Using kernels we can circumvent this problem.

A *kernel function* $k$ is defined as $k: \mathbb{R}^N\times \mathbb{R}^N \rightarrow \mathbb{R}$
Some problems can be reformulated in terms of a kernel function (c.f. Bishop 6.1).
$$k(x_i, x_j) \defeq \phi(x_i)^T\phi(x_j) $$
It turns out that in those cases, we can simply choose any kernel, some of which implicitly use an "infinite-dimensional" feature space.

> **Task 5** Find out how this is possible by calculating the feature transformation $\phi(x)$ corresponding to the kernel: 
> $$k(x_1, x_2) = \frac{1}{1-x_1x_2}, \text{ with } x_1, x_2 \in (0, 1)$$
Note: $x_1$ and $x_2$ are simply 1d vectors / scalars.




To come up with new kernels, it is possible to combine kernels in various ways (cf. Bishop 6.2 "Techniques for Constructing New Kernels").

> **Task 6** Show that for $N\in\mathbb{N}$ and $a_i \ge 0$ for $i\in\{0, ..., N\}$ the following function $k$ is a valid kernel.
>
> $$k(x_1, x_2) = \sum_{i=1}^{N}a_i\left(x_1^T x_2\right)^i + a_0, \text{ with } x_1, x_2 \in\mathbb{R}^d$$




A commonly used kernel is the Gaussian kernel:
$$k(\textrm{x}, \textrm{x}^\prime) = \textrm{exp}\left(- \frac{||\textrm{x} - \textrm{x}^\prime||^2}{2\sigma^2}\right)$$

> **Task 7** Show that the gaussian kernel can be expressed as the inner product of an infinite-dimensional feature vector.
> 
> *Hints*
> * Start by transforming the kernel into a product of three exponentials 
$$k(\x, \xprime) = \exp(\ldots)\exp(\ldots)\exp(\ldots)$$
> * Then consider the univariate case $k(x, x^\prime)$ where $x$ and $x^\prime$ are scalars. Use the definition of the exponential function to expand the middle exponential into an infinite sum.
Only consider the middle exponential term for now.
I.e. transform $k(x, x^\prime) = \exp\left(\frac{x x^\prime}{\sigma^2}\right)$ into an infinite sum, then into a inner product of vectors $\phi(x)^T\phi(x^\prime)$
> * Finally, use your derived $\phi(x)$ and the kernel techniques in Bishop 6.2 ("Techniques for Constructing New Kernels") to show that the gaussian kernel can be expressed as an inner product of an infinite-dimensional feature vector.


